In [25]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
import json
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2023-12-01-preview"
)

In [28]:
def search_knowlegde_base(input):
    """ This should execute a RAG search """
    print(input)
    return None

sytem_message = """
- You are an assistant in a company.
- You are helping a user with a question. 
- If the user asks a question that is specific to their company, you can search the knowledge base for an answer using the function search_knowlegde_base.
- Make sure to use an input query that can be used as a semantic search query.
- If the search_knowlegde_base function returns an empty string then you MUST say you can't find the information. Don't make anything up.

Examples:
- If the user asks 'What kind of roles are available in my company?', you can use 'roles available' as the input query.
- If the user asks 'How do I do summarize two columns in Excel? 'you should NOT use the search_knowlegde_base function.
"""

def run_conversation(input: str):
    # Step 1: send the conversation and available functions to the model
    messages = [
        {"role": "system", "content": sytem_message},
        {"role": "user", "content": input},
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "search_knowlegde_base",
                "description": "Searches the knowledge base for an answer to the user's question using generated embeddings from the users input. Returns the answer if found, or null if not found. ",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "input": {
                            "type": "string",
                            "description": "The user's input to search for an answer to.",
                        }
                    },
                    "required": ["input"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-4",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "search_knowlegde_base": search_knowlegde_base,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                input=function_args.get("input"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response or "",
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
    else:
        return response

In [29]:
sample_questions = [
    "How many wheels does a car have?",
    "What does our employee handbook say about vacation?",
    "How do I summarize two columns in Excel?",
    "Show me how to multiple two matrices in Python."
]

for q in sample_questions:
    res = run_conversation(q)
    print(res.choices[0].message)
    print(res.choices[0].message.content)
    

ChatCompletionMessage(content='A typical car has four wheels.', role='assistant', function_call=None, tool_calls=None)
A typical car has four wheels.
employee handbook vacation policy
ChatCompletionMessage(content="I'm sorry, I couldn't find any information about the vacation policy in the employee handbook in our company knowledge base. You may want to reach out to your HR department for more accurate information.", role='assistant', function_call=None, tool_calls=None)
I'm sorry, I couldn't find any information about the vacation policy in the employee handbook in our company knowledge base. You may want to reach out to your HR department for more accurate information.
ChatCompletionMessage(content='To summarize two columns in Excel, you can use various functions depending on the type of summary you need. Here are a few examples:\n\n1. Sum: `=SUM(A2:B7)` will return the sum of all numbers in columns A and B from the 2nd row to the 7th.\n\n2. Average: `=AVERAGE(A2:B7)` will return the